#   **Group Report**

### 1. **Installation and imports of all the necessary libraries**

In [199]:
# In case required, please uncomment and run the code:
# %pip install mesa networkx pandas matplotlib

import random, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

import mesa
from mesa import Model, Agent
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

print("Mesa version:", mesa.__version__)  # should be 3.x

Mesa version: 3.3.0


### 2. **Utilities and Configuration**

+ We keep the skills bounded and present our constants here. 

In [200]:
def bound(x, lower=0.0, higher=100.0):
    # Bound the skills value between [lower, higher].
    return max(lower, min(x, higher))

# Defaults for the model to experiment: 

ATTENDANCE_PROBABILITY = 0.60
CLASS_DURATION_MINUTES = [45, 60, 75, 90]
SELF_STUDY_THRESHOLD = 0.04
TEACHER_FEEDBACK_PROBABILITY = 0.45

### 3. **Agent Architecture** 

####    a. Agent Types 

####    b. Agent Properties

####    c. Behavioral Patterns

PublicPlaces Class

+ Represents study locations (libraries, cafes, work rooms) with varying resource quality and capacity.

In [201]:
# 1. Agent : Place 

class PublicPlaces(Agent):
    def __init__(self, model, type, resource_quality=0.5, capacity=7):
        super().__init__(model)
        self.type = type 
        self.resource_quality = float(resource_quality)
        self.capacity = int(capacity)
        self.current_learners = set()
        
    def space_left(self) -> int:
        """Remaining capacity in the public place."""
        return max(0, self.capacity - len(self.current_learners))

    def reset(self) -> None:
        """Clear out the learners for the new simulation day."""
        self.current_learners = set()
        
    def step(self):
        # Place is passive and influence happens via its attributes.
        pass


Teacher Class

+ Represents instructor who provides teaching and feedback to students.

In [202]:
# 2. Agent : Teacher

class Teacher(Agent):
    
    def __init__(self, model, teaching_quality=0.7, feedback_level=1.5, classroom_capacity=25):
        super().__init__(model)
        self.teaching_quality = float(teaching_quality)
        self.feedback_level = float(feedback_level)
        self.classroom_capacity = int(classroom_capacity)
        
    def step(self):
        pass


Student Class 

+ Primary active agent rep learners with individual characteristics and behaviors.

Core attributes:

+ skill: Initial value uniform random [10, 50]
+ learning_rate: Personal learning speed multiplier
+ learning_style: "social" or "solo" preference
+ access_to_resources: Boolean for resource availability
+ energy: Energy level (currently unused)

Tracking counters:

+ attendance: Total classes attended
+ in_class_duration: Cumulative time in class (minutes)
+ teacher_interactions: Count of feedback sessions

Behavioral parameters:

+ attendance_probability: Likelihood of attending (default 0.75)
+ class_duration_minutes: Random choice from [45, 60, 75, 90]


In [203]:
# 3. Agent : Student

class Student(Agent):
    
    def __init__(self, model, learning_rate=1.0, learning_style="social", access_to_resources=True):
        super().__init__(model)
        
        # class attributes
        
        self.skill = self.model.random.uniform(10, 50)
        self.learning_rate = float(learning_rate)
        self.learning_style = learning_style  
        self.access_to_resources = bool(access_to_resources)
        self.energy = 0
        
        # counters 
        
        self.attendance = 0
        self.in_class_duration = 0
        self.teacher_interactions = 0
        
        self.attendance_probability = ATTENDANCE_PROBABILITY
        self.class_duration_minutes = self.model.random.choice(CLASS_DURATION_MINUTES)
        
    def step(self):
        self.attend_class()
        place = self.select_study_place()
        self.prefer_study(place)
        
    # --- Helper Classes ---
    
    # --- 1. Class Attendance ---

    def attend_class(self):
        """
        
        """
        if self.model.random.random() < self.attendance_probability:
            # If a student skips a class, we set up a threshold value that they are self-studying at home/any place and give them a relatively small score.
            self.skill = bound(self.skill + 0.025 * self.learning_rate) # this 0.025 can be adjusted later, either as a constant 
            return False
        
        # 1. Measuring the attendance 
        self.attendance += 1
        min = self.model.random.choice(CLASS_DURATION_MINUTES)
        self.in_class_duration += min
        duration_points = 2 if min >= 60 else 0 
        
        # 2. Learning from teacher 
        teacher = self.model.teachers[0] if self.model.teachers else None
        teaching_quality_measure = teacher.teaching_quality if teacher else 0.5
        learning_gain = 0.3 * teaching_quality_measure * self.learning_rate
        
        # 3. Class Points 
        skill_level_points = 3 if self.skill > 33.3 else (5 if self.skill < 66 else 7) # Skill level: low, medium, high
        learning_speed = 3 if self.learning_rate < 0.75 else (5 if self.learning_rate < 1.25 else 7) # Learning speed: slow, medium, fast
        attendance_points = 1 + duration_points
        
        # add everything together - all the gains
        self.skill = bound(self.skill + learning_gain + 0.02 * (skill_level_points + learning_speed + attendance_points))
        
        if self.model.random.random() < TEACHER_FEEDBACK_PROBABILITY:
            """ 
            Condition: We estimate that 45% of the students who attend the class interact with the teacher and ask for feedback on that particular day. 
            
            1. Get a random number between 0 and 1 in the students group to simulate teacher interactions. 
            2. 45% of the time, we model that this condition is True.
            
            It becomes a probabilistic decision, hence this randomness simulates some real classroom scenarios. 
            As not every student queues for teacher-feedback in class every day.
            """
            self.model.class_lottery.append(self.unique_id) #
        
        return True 
        
    # --- 2. Teacher Feedback ---
    
    # This method explains what happens when a student gains feedback from the teacher.
    
    def gain_teacher_feedback(self, teacher):
        """
        pass - need to add docstring later
        """
        self.teacher_interactions += 1 # counter which tracks how many times a student has received teacher feedback.
        feedback_effect = 0.8 * teacher.teaching_quality * teacher.feedback_level * self.learning_rate        
        self.skill = bound(self.skill + feedback_effect + 0.06 * 3)
        
    # --- 3. Selecting a study space (deciding where to study) ---
    
    def select_study_place(self):
        """ 
        Select a public place (library, cafe, or work rooms).
        """
        available_spaces = []
        for space in self.model.study_spaces:
            if space.space_left() > 0:
                social_bonus = 0.2 if self.learning_style == "social" else 0.0
                util = space.resource_quality + social_bonus + self.model.random.uniform(-0.05, 0.05)
                available_spaces.append((util, space))
        
        # we set a probability that there is a 15% chance that a student stays at home or if no spaces are available.
        if not available_spaces or self.model.random.random() < 0.15:
            return None  # Study at home
        
        # sort by utility and choose the best one (20% exploration factor)
        available_spaces.sort(key=lambda x: x[0], reverse=True)
        if self.model.random.random() > 0.2:
            chosen = available_spaces[0][1] # choose the best option
        else:
            chosen = self.model.random.choice(available_spaces[:min(3, len(available_spaces))])[1]  
        
        self.model.grid.move_agent(self, chosen.pos)
        chosen.current_learners.add(self.unique_id)
        return chosen
    
    # --- 4. Study Session (either solo or social learning) ---
    
    def prefer_study(self, place):
        # If there is no place to study, the student studies at home + small threshold gain.
        if place is None: 
            self.skill = bound(self.skill + 0.05 * self.learning_rate)
            return 
        
        # Base solo learning gain by resource quality 
        solo_gain = 0.15 * self.learning_rate * place.resource_quality
        
        # If social learner and student is social type
        if self.model.social_learning and self.learning_style == "social":
            peers_here = [sid for sid in place.current_learners if sid !=self.unique_id]
            if peers_here:
                k = min(3, len(peers_here))
                selected_peers = self.model.random.sample(peers_here, k)
                peer_boost = 0.0
                
                for pid in selected_peers:
                  w = self.model.get_tie_strength(self.unique_id, pid)
                  peer_skill = self.model.get_agent_skill(pid)
                  gap = max(0.0, peer_skill - self.skill)  # only learn from stronger peers
                  peer_boost += 0.08 * w * (gap / 100.0) * self.learning_rate

                # Combined solo + peer effects
                self.skill = bound(self.skill + solo_gain + peer_boost)
                return

        # Otherwise, solo study only
        self.skill = bound(self.skill + solo_gain)
                

### Short explanation 


####    **attend_class()**

Simulates class attendance and learning from instruction. 

1. Attendance check: 75% probability of attending
- If skip: Gain minimal self-study (0.025 × learning_rate), return False

2. If attend:
- Increment attendance counter
- Select class duration (45-90 minutes)
- Award duration points: 2 if ≥60 min, else 0

3. Learning from teacher:
- Get teacher's teaching quality (default 0.5 if no teacher)
- Calculate: learning_gain = 0.3 × teaching_quality × learning_rate

4. Class participation points:
- Skill level: Low (>33) = 3pts, Medium (<66) = 5pts, High = 7pts
- Learning speed: Slow (<0.75) = 3pts, Med (<1.25) = 5pts, Fast = 7pts
- Attendance: 1 + duration_points

5. Skill update:
- skill += learning_gain + 0.02 × (skill_pts + speed_pts + attend_pts)
- Bounded to [0, 100]

6. Teacher feedback lottery:
- 45% chance to enter feedback queue
- Adds student ID to model.class_lottery


####    **gain_teacher_feedback**

This applies personal feedback boost from teacher interaction.


- Increment teacher_interactions counter
- Calculate feedback effect:
    - feedback_effect = 0.8 × teaching_quality × feedback_level × learning_rate
- Apply bonus: skill += feedback_effect + 0.18 (0.06 × 3)
- Bound skill to [0, 100]


####    **select_study_place**

This function provides a student's decision-making when choosing where to study outside of class. Models rational choice with bounded rationality (balancing optimal choices with exploration).

Select a public place (library, cafe, or work rooms) for studying.
      
Decision process:

1. Evaluate all available spaces with capacity
2. Calculate utility scores based on resource quality and learning style
3. Consider staying home (15% probability)
4. Balance exploitation (best choice) vs exploration (trying alternatives)
5. Move to chosen location and register presence
    
Returns:
- PublicPlaces object if space chosen, None if staying home

####    **study()**

This function simulates actual learning that occurs during a study session. Models both individual (solo) learning and social (peer-based) learning dynamics. 

Learning modes:
- Home study: Minimal baseline gain (no resources)
- Solo study: Resource-quality dependent learning
- Social study: Solo gain + peer learning boost (if social learner)

#### cognitive theories we can actually talk about are: 

  1. Social Learning Theory (Bandura): Learning through peer observation
  2. Zone of Proximal Development (Vygotsky): Learning from slightly more
  skilled peers
  3. Bounded Rationality (Simon): Satisfising rather than optimizing
  4. Social Capital Theory: Network ties affect knowledge transfer


#### YET TO DO; 

#### Step 3 (main stage): Build the Mesa Model (Mesa 3 activation, no need for RandomActivation) - why no random because it was replaced with AgentSets you can use shuffle probably. 

+ Environment: create a grid campus (MultiGrid) with your public study spaces (capacity + resource quality) and places teachers and students on it.
+ Network: builds a social network over students (e.g., ring/ER/small-world) and assigns tie strengths (used in peer learning).
+ Scenarios: boolean flag social_learning toggles Scenario 1 (solo) vs Scenario 2 (social).

Time step (daily):
- reset rooms & lottery
- activate all agents once in random order
- resolve teacher feedback (limited capacity)
- collect weekly measures.

+ Measures: a DataCollector records weekly mean/median skill, total attendance, teacher interactions, and per-agent skill.

+ Experiments-ready: parameters like capacities, feedback_level, teaching_quality, network type, and social_learning

### next steps (these are quite easy if we can design the multi-agent systems perfectly till previous step, these are just experiment methods i think could be good.)

#### a. Scenario runs (answers Driving Q1)

- Run Scenario 1 with social_learning=False (individual learning only).
- Run Scenario 2 with social_learning=True (adds peer effect).
- Extract weekly model metrics; plot weekly mean skill for both.
- Extract final per-agent skills; compare distributions (e.g., histograms).

+ Why: direct evidence for/against the social-learning advantage.

####   b. Replications (meets course reqs)

- Write a small function to replicate any configuration ≥10 times with different seeds.
- For a base configuration (e.g., small-world, default places), replicate solo and social each ≥10 times.
- Summarize mean ± SD of final mean skill (optionally run a simple Welch’s t-stat).

+ Why: robust results and meets the “replicate at least 10 times” requirement.

####   c. results summary - we can simplify and keep the metrics as he mentioned it in one of his lectures on descriptive stats or sth

- Define a helper to scale place capacity (e.g., ×0.5, ×1.0, ×1.5).

- For each run, compute: (this is good idea)
1. Weekly curves and AUC (area under weekly mean-skill curve).
2. Final mean skill.
3. Differences (Social − Solo) for AUC and final mean.

- Assemble a tidy DataFrame; visualize:
1. Horizontal bar chart of AUC advantage (sorted).
2. Three illustrative weekly curves (best / median / worst).

Save weekly outputs (CSV) for solo/social.

Save sweep summary (CSV) with AUC differences and final means.

Export plots as PNG to include in the Results section.